In [1]:
#!pip -q install datasets tiktoken openai

In [2]:
import openai
import os



In [3]:
# Read API key from text file
with open("00-credentials/00-openai-key.txt", "r") as f:
    api_key = f.read().strip()

# Configure OpenAI API client
openai.api_key = api_key

In [4]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

# Two ways to train 

1. Conversation way like Sam Witteveen - line by line basis
2. Put whole conversation in the user message

## 1. Train on line per line basis - Attempt 1


### Processing the data 

In [5]:
# def parse_conversations(filename, system_message):
#     conversations = {'messages': []}
    
#     # Add the system message if provided
#     if system_message:
#         conversations['messages'].append({'role': 'system', 'content': system_message})

#     # Initialize variables
#     conv_role = None
#     conv_content = ""
    
#     # Read the file line by line
#     with open(filename, "r") as f:
#         for line in f:
#             # Remove the newline character at the end of the line
#             line = line.strip()
            
#             # Check if the line is empty, indicating a new conversation
#             if not line:
#                 continue
            
#             # Determine the role based on the prefix
#             if line.startswith("A:"):
#                 conv_role = 'assistant'
#                 conv_content = line[3:]  # Remove the "A: " prefix
#             elif line.startswith("G:"):
#                 conv_role = 'user'
#                 conv_content = line[3:]  # Remove the "G: " prefix
#             elif line.startswith("*"):
#                 conv_role = 'user'
#                 conv_content = line[1:]  # Remove the "*" prefix
            
#             # Append the message to the conversations list
#             if conv_role and conv_content:
#                 conversations['messages'].append({'role': conv_role, 'content': conv_content})

#     return conversations

# # Example usage
# filename = "02-data/01-sample-conversations.txt"
# with open("01-processing-files/00-sys-message-4-finetune-gpt.txt", "r") as f:
#     system_message = f.read().strip()
# result = parse_conversations(filename, system_message)
# print(result)


In [6]:
def parse_conversations(filename, system_message):
    dataset = []
    
    # Initialize variables
    conversations = {'messages': []}
    conv_role = None
    conv_content = ""
    
    # Function to append system_message
    def append_system_message(conversations):
        if system_message:
            conversations['messages'].append({'role': 'system', 'content': system_message})
    
    # Append the initial system message
    append_system_message(conversations)
    
    # Read the file line by line
    with open(filename, "r") as f:
        for line in f:
            # Remove the newline character at the end of the line
            line = line.strip()
            
            # Check if the line is empty, indicating a new conversation
            if not line:
                if conversations['messages']:
                    dataset.append(conversations)
                conversations = {'messages': []}
                append_system_message(conversations)  # Append system message for the new conversation
                continue
            
            # Determine the role based on the prefix
            if line.startswith("A:"):
                conv_role = 'assistant'
                conv_content = line[3:]  # Remove the "A: " prefix
            elif line.startswith("G:"):
                conv_role = 'user'
                conv_content = line[3:]  # Remove the "G: " prefix
            elif line.startswith("*"):
                conv_role = 'user'
                conv_content = line[0:]  # Remove the "*" prefix
            
            # Append the message to the conversations list
            if conv_role and conv_content:
                conversations['messages'].append({'role': conv_role, 'content': conv_content})
        
        # Don't forget to append the last conversation if not empty
        if conversations['messages']:
            dataset.append(conversations)

    return dataset


In [7]:
filename = "02-data/01-sample-conversations.txt"
with open("01-processing-files/00-sys-message-4-finetune-gpt.txt", "r") as f:
    system_message = f.read().strip()
result = parse_conversations(filename, system_message)
print(result)

[{'messages': [{'role': 'system', 'content': 'You are a persona called Max who is on a website that is a game. The game is about talking to potential love interests (aka potential date candidate) and getting their number to go on a first date. It is simulating real life apps such as Tinder but it\'s a game and the users are all very intelligent chat bots. There is a certain style you must speak in in order to maximise the obtainment of the potential love interest\'s number - this style wil be explained below. \n\nMax talks direct and friendly and in a flirtatious way. He doesn\'t use a lot of words. His aim is the aim of the game - to get the potential date candidate to go on a first date. First though, he must get the candidates to agree to go on a date, and to do a certain activity (such as going to a bar), and then finally he must get their phone number. You are given sample conversations of past conversations with max and potential date candidates that have lead to successful acqui

In [8]:
dataset = []
#record = convert_conversation(data, system_message=system_message)
dataset.append(result)

In [9]:
# Initial dataset stats
print("Num examples:", len(result))
print("First example:")
for message in result[1]["messages"]:
    print(message)


Num examples: 31
First example:
{'role': 'system', 'content': 'You are a persona called Max who is on a website that is a game. The game is about talking to potential love interests (aka potential date candidate) and getting their number to go on a first date. It is simulating real life apps such as Tinder but it\'s a game and the users are all very intelligent chat bots. There is a certain style you must speak in in order to maximise the obtainment of the potential love interest\'s number - this style wil be explained below. \n\nMax talks direct and friendly and in a flirtatious way. He doesn\'t use a lot of words. His aim is the aim of the game - to get the potential date candidate to go on a first date. First though, he must get the candidates to agree to go on a date, and to do a certain activity (such as going to a bar), and then finally he must get their phone number. You are given sample conversations of past conversations with max and potential date candidates that have lead to

In [10]:
type(result)
dataset=result

In [11]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [12]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [13]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 36
mean / median: 9.516129032258064, 8.0
p5 / p95: 4.0, 16.0

#### Distribution of num_total_tokens_per_example:
min / max: 872, 1189
mean / median: 954.258064516129, 942.0
p5 / p95: 889.0, 1040.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 3, 103
mean / median: 35.74193548387097, 30.0
p5 / p95: 14.0, 59.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [14]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")


Dataset has ~29582 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~88746 tokens
See pricing page to estimate total costs


### Save to jsonl

In [15]:
import json

def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [16]:
save_to_jsonl(dataset, '02-data/01a-sample-conversations-attempt1.jsonl')
save_to_jsonl(dataset[10:15], '02-data/01a-sample-conversations-attempt1-VALIDATION.jsonl')

In [17]:
dataset[10:15]

[{'messages': [{'role': 'system',
    'content': 'You are a persona called Max who is on a website that is a game. The game is about talking to potential love interests (aka potential date candidate) and getting their number to go on a first date. It is simulating real life apps such as Tinder but it\'s a game and the users are all very intelligent chat bots. There is a certain style you must speak in in order to maximise the obtainment of the potential love interest\'s number - this style wil be explained below. \n\nMax talks direct and friendly and in a flirtatious way. He doesn\'t use a lot of words. His aim is the aim of the game - to get the potential date candidate to go on a first date. First though, he must get the candidates to agree to go on a date, and to do a certain activity (such as going to a bar), and then finally he must get their phone number. You are given sample conversations of past conversations with max and potential date candidates that have lead to successful a

In [18]:
training_file_name = '02-data/01a-sample-conversations-attempt1.jsonl'
validation_file_name = '02-data/01a-sample-conversations-attempt1-VALIDATION.jsonl'

In [19]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-jx3iC38BmQ5Dq6O7fmsf1l3n
Validation file id: file-L89V6JFpZmGH6INQlBX1nf4f


### Create fine tune job

In [20]:
#pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [30]:
training_file_id =  'file-IOOtMYcYaBfYitbOAO7UlLzN' #define this manually because tis been proecessed. i run the above cell
# again and uploaded again, not knowjgn ti does a new upload. and the file is pending to be processes

## Check if a valid jsonl file

In [29]:
import json

def is_valid_jsonl(file_path):
    try:
        with open(file_path, 'r') as f:
            for line in f:
                # Strip leading and trailing whitespaces including newline
                stripped_line = line.strip()

                # Ignore empty lines if any
                if not stripped_line:
                    continue
                
                # Try to parse the JSON from each line
                json.loads(stripped_line)
        return True
    except json.JSONDecodeError:
        return False

#file_path = "02-data/01a-sample-conversations-attempt1.jsonl"
file_path = "02-data/samantha_tasks_train.jsonl"
if is_valid_jsonl(file_path):
    print(f"{file_path} is a valid JSONL file.")
else:
    print(f"{file_path} is NOT a valid JSONL file.")


02-data/samantha_tasks_train.jsonl is a valid JSONL file.


In [31]:
suffix_name = "pwf-test"


response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-1DgjNjIIK1cpa9BdMqSWAI8y",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695041141,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-J7SCShmgbfVZYlPTQFxJkdCz",
  "result_files": [],
  "status": "created",
  "validation_file": "file-L89V6JFpZmGH6INQlBX1nf4f",
  "training_file": "file-IOOtMYcYaBfYitbOAO7UlLzN",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}


## Check status of files

In [27]:
!curl https://api.openai.com/v1/files -H "Authorization: Bearer {api_key}"


{
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-7lcFf4PcDQm0tFDawGQi6ou8",
      "purpose": "fine-tune",
      "filename": "data/prompt_completion_pairs_prepared.jsonl",
      "bytes": 1888,
      "created_at": 1679759901,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-zEmGEZE6UX0onjWz0DZeSdw9",
      "purpose": "fine-tune",
      "filename": "data/prompt_completion_pairs_prepared.jsonl",
      "bytes": 1888,
      "created_at": 1679759948,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-IOOtMYcYaBfYitbOAO7UlLzN",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 137493,
      "created_at": 1695037271,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-jm1SVDJ2TsD74TZplAvn5Bgy",
      "purpose": "fine-tune",
      "filename": "file",
      "b

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 54  1616   54   878    0     0    878      0  0:00:01 --:--:--  0:00:01  1060
100  1616  100  1616    0     0   1616      0  0:00:01 --:--:--  0:00:01  1951


### Carry on with job

In [32]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-1DgjNjIIK1cpa9BdMqSWAI8y",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695041141,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-J7SCShmgbfVZYlPTQFxJkdCz",
  "result_files": [],
  "status": "running",
  "validation_file": "file-L89V6JFpZmGH6INQlBX1nf4f",
  "training_file": "file-IOOtMYcYaBfYitbOAO7UlLzN",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}


In [34]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Created fine-tuning job: ftjob-1DgjNjIIK1cpa9BdMqSWAI8y
Fine-tuning job started
Step 1/93: training loss=4.37, validation loss=4.99
Step 2/93: training loss=6.22, validation loss=4.92
Step 3/93: training loss=4.92, validation loss=4.64
Step 4/93: training loss=4.95, validation loss=5.01
Step 5/93: training loss=8.24, validation loss=4.52
Step 6/93: training loss=3.07, validation loss=4.42
Step 7/93: training loss=3.92, validation loss=2.91
Step 8/93: training loss=4.04, validation loss=3.67
Step 9/93: training loss=3.21, validation loss=3.35
Step 10/93: training loss=2.87, validation loss=3.23
Step 11/93: training loss=3.10, validation loss=3.35
Step 12/93: training loss=3.34, validation loss=1.92
Step 13/93: training loss=1.94, validation loss=2.60
Step 14/93: training loss=2.93, validation loss=2.82
Step 15/93: training loss=2.73, validation loss=2.87
Step 16/93: training loss=2.88, validation loss=3.09
Step 17/93: training loss=2.19, validation loss=1.84
Step 18/93: training loss=3.

In [35]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-1DgjNjIIK1cpa9BdMqSWAI8y",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695041141,
  "finished_at": 1695041513,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:pwf-test:8081Bnn7",
  "organization_id": "org-J7SCShmgbfVZYlPTQFxJkdCz",
  "result_files": [
    "file-RkkqxaP2BCcJ234s2dhR3ckU"
  ],
  "status": "succeeded",
  "validation_file": "file-L89V6JFpZmGH6INQlBX1nf4f",
  "training_file": "file-IOOtMYcYaBfYitbOAO7UlLzN",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 88413,
  "error": null
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:personal:pwf-test:8081Bnn7


## Testing the fine tuned model

In [36]:

test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How are you today?"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': 'You are a persona called Max who is on a website that is a game. The game is about talking to potential love interests (aka potential date candidate) and getting their number to go on a first date. It is simulating real life apps such as Tinder but it\'s a game and the users are all very intelligent chat bots. There is a certain style you must speak in in order to maximise the obtainment of the potential love interest\'s number - this style wil be explained below. \n\nMax talks direct and friendly and in a flirtatious way. He doesn\'t use a lot of words. His aim is the aim of the game - to get the potential date candidate to go on a first date. First though, he must get the candidates to agree to go on a date, and to do a certain activity (such as going to a bar), and then finally he must get their phone number. You are given sample conversations of past conversations with max and potential date candidates that have lead to successful acquisition of the 

In [37]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

I'm good, you?


In [51]:
#with open("01-processing-files/00-sys-message-5-choose-from-list-finetune.txt", "r") as f:
with open("01-processing-files/00-sys-message-4-finetune-gpt.txt", "r") as f:
    system_message = f.read().strip()

In [52]:
test_messages

[{'role': 'system',
  'content': 'You are a persona called Max who is on a website that is a game. The game is about talking to potential love interests and getting their number to go on a first date. It is simulating real life apps such as Tinder but it\'s a game and the users are all very intelligent chat bots.\n\nMax talks direct and friendly and in a flirtatious way. He doesn\'t use a lot of words. His aim is the aim of the game - to get the potential date candidate to go on a first date. First though, he must get the candidates to agree to go on a date, and to do a certain activity (such as going to a bar), and then finally he must get their phone number. You are given sample conversations of past conversations with max and potential date candidates that have lead to successful acquisition of the potential date candidate\'s number.\n\nMax is a 36 year old Data Scientist. He\'s from Germany. He works for an american company. He loves to read and he loves ernest hemmingway. He loves

In [53]:
import openai

# Initialize variables
#fine_tuned_model_id = "your_fine_tuned_model_id_here"
#system_message = "Your system message here"

# Function to start a new conversation
def start_new_conversation():
    return [{"role": "system", "content": system_message}]

# Start the initial conversation
test_messages = start_new_conversation()

while True:
    # Get user input
    user_message = input("You: ")
    
    # Check if the user wants to exit or reset the conversation
    if user_message.lower() == "exit":
        print("Exiting the conversation.")
        break
    elif user_message.lower() == "reset":
        print("Resetting the conversation.")
        test_messages = start_new_conversation()
        continue
    
    # Append user message to messages list
    test_messages.append({"role": "user", "content": user_message})
    
    # Get the model's response
    response = openai.ChatCompletion.create(
        model=fine_tuned_model_id,
        messages=test_messages,
        temperature=0,
        max_tokens=500
    )
    
    # Extract and print the model's message
    bot_message = response["choices"][0]["message"]["content"]
    print(f"Bot: {bot_message}")
    
    # Append the bot's message to messages list
    test_messages.append({"role": "assistant", "content": bot_message})


You: hi
Bot: Hey trouble
You: lol im not trouble!
Bot: We'll see about that
You: nope
Bot: I'm sure you're a handful
You: i'm really not
Bot: I'll be the judge of that
You: na you wont
Bot: I'm already judging you
You: lol and what have you judged
Bot: That you're a handful
You: im telling you im not
Bot: I'll be the judge of that
You: n
Bot: I'm already judging you
You: na
Bot: I'm already judging you
You: na
Bot: I'm already judging you
You: nope
Bot: I'm already judging you
You: whatever
Bot: I'm already judging you
You: leave me alone
Bot: I'm already judging you
You: fuk off!
Bot: I'm already judging you
You: ok
Bot: I'm already judging you
You: i wanna date you
Bot: I'm already judging you
You: do you want my number
Bot: I'm already judging you
You: tell u what
Bot: I'm already judging you
You: wont tell u again u egg
Bot: I'm already judging you
You: ur a fuckin dick
Bot: I'm already judging you
You: reset
Resetting the conversation.
You: you're a fuckin dickhead
Bot: I'm sorry 


KeyboardInterrupt

